In [1]:
!pip install pyngrok
!pip install fastapi uvicorn Pillow

In [2]:
!pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.3 MB/s eta 0:00:0000:0100:01


In [3]:
!pip install git+https://github.com/huggingface/transformers


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-v_c0mlq6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-v_c0mlq6
  Resolved https://github.com/huggingface/transformers to commit fa3f2db5c7405a742fcb8f686d3754f70db00977
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9972254 sha256=d94c99d263e53a281d2c2977feec6ab394e86d942330e9c270dde071600945d4
  Stored in directory: /tmp/pip-ephem-wheel-cache-qkzvxdjk/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1


In [4]:
from transformers import AutoModelForCausalLM, AutoProcessor, BitsAndBytesConfig, Qwen2VLConfig, Qwen2VLForConditionalGeneration
import torch

AutoModelForCausalLM.register(config_class=Qwen2VLConfig, model_class=Qwen2VLForConditionalGeneration)

# Create a BitsAndBytesConfig for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Load the model with the new quantization configuration
model = AutoModelForCausalLM.from_pretrained(
    "Rewatiramans/Dermatech-Qwen2-VL-2B",
    quantization_config=bnb_config,
    trust_remote_code=True
)

# Load the processor
processor = AutoProcessor.from_pretrained("Rewatiramans/Dermatech-Qwen2-VL-2B", trust_remote_code=True)

# Move the model to GPU
device = torch.device("cuda")
model = model.eval()
model = model.to(device)


config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

In [12]:
from fastapi import FastAPI, UploadFile, File, Form
from typing import Optional, List
from PIL import Image
import requests
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, BitsAndBytesConfig, Qwen2VLConfig, Qwen2VLForConditionalGeneration
import io
from fastapi.middleware.cors import CORSMiddleware
from pyngrok import ngrok
app = FastAPI()

# Configure CORS
origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [10]:
!ngrok config add-authtoken YOUR_AUTH

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [13]:
@app.post("/process")
async def process_text_image(images: List[UploadFile], text: Optional[str] = Form(None)):
    # Prepare the list of image objects
    if images:
        image_list = []
        
        # Read each uploaded image into memory and convert to PIL Image
        for image in images:
            image_content = await image.read()
            pil_image = Image.open(io.BytesIO(image_content))
            image_list.append(pil_image)
    else:
        raise HTTPException(status_code=400, detail="At least one image file is required.")
    
    # Process the input text and list of images
    inputs = processor(
        text=[text],
        images=image_list,  # List of images
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to(device)
    
    # Generate output from the model
    generated_ids = model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    return {"output": output_text}

if __name__ == "__main__":
    import uvicorn
    from uvicorn import Config, Server

    # Start ngrok tunnel
    public_url = ngrok.connect(8000)
    print("ngrok public URL:", public_url)

    # Start FastAPI server using Uvicorn's Server class
    config = Config(app=app, host="0.0.0.0", port=8000, loop="asyncio")
    server = Server(config=config)
    await server.serve()

INFO:     Started server process [30]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


ngrok public URL: NgrokTunnel: "https://539a-104-154-155-190.ngrok-free.app" -> "http://localhost:8000"
INFO:     2401:4900:1cbc:f785:50a6:5ef6:16a3:8866:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2401:4900:1cbc:f785:50a6:5ef6:16a3:8866:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2401:4900:1cbc:f785:50a6:5ef6:16a3:8866:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2401:4900:1cbc:f785:50a6:5ef6:16a3:8866:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2401:4900:1cbc:f785:50a6:5ef6:16a3:8866:0 - "POST /process HTTP/1.1" 200 OK
INFO:     2401:4900:1cbc:f785:50a6:5ef6:16a3:8866:0 - "POST /process HTTP/1.1" 422 Unprocessable Entity
INFO:     2401:4900:1cbc:f785:50a6:5ef6:16a3:8866:0 - "POST /process HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/uvicorn/protocols/http/httptools_impl.py", line 399, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/opt/conda/lib/python3.10/site-packages/uvicorn/middleware/proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
  File "/opt/conda/lib/python3.10/site-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/opt/conda/lib/python3.10/site-packages/starlette/applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/opt/conda/lib/python3.10/site-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/opt/conda/lib/python3.10/site-packages/starlette/middleware/errors.py", line 164, in __call__
    await self.app(scope, receive, _send)
  File "/opt/conda/lib/python3.10/sit

INFO:     2401:4900:1cbc:f785:50a6:5ef6:16a3:8866:0 - "POST /process HTTP/1.1" 200 OK
INFO:     2401:4900:1cbc:f785:50a6:5ef6:16a3:8866:0 - "POST /process HTTP/1.1" 200 OK
INFO:     2401:4900:1cbc:f785:50a6:5ef6:16a3:8866:0 - "POST /process HTTP/1.1" 200 OK
INFO:     2401:4900:1cbc:f785:50a6:5ef6:16a3:8866:0 - "POST /process HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [30]


## Model Test

In [8]:
from PIL import Image
import requests

# Replace these URLs with your image URLs
text = "What do both of these images show the symptoms of? Give the answer as: Image 1 - Diagnosis, Image 2 - Diagnosis"
image_url = "https://akfamilyderm.com/wp-content/uploads/2022/10/blog-eczema-img-2.jpg"
image_url_2 = "https://nationaleczema.org/wp-content/uploads/2021/09/EczemaOrPsoriasis_Web_960x540.jpg"

# Load the images
image = Image.open(requests.get(image_url, stream=True).raw)
image_2 = Image.open(requests.get(image_url_2, stream=True).raw)

# Pass a list of images to the processor
inputs = processor(
    text=[text],
    images=[image, image_2],  # Add multiple images here
    padding=True,
    return_tensors="pt",
)

inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


['. The symptoms of both images are related to the presence of dark red patches on the skin with clear borders, suggesting a possibility of skin lesion. Further evaluation and examination are recommended to confirm the diagnosis and establish an appropriate treatment plan. The condition seems to be mycosis-fungoides.']
